In [18]:
import tensorflow as tf
import pandas as pd
import numpy as np
import keras

In [28]:
path = '/home/murilo/workspace/tcc/tcc/python_model/src'
path_to_save = '/home/murilo/workspace/tcc/tcc/python_model/src/modelos_micro'

In [36]:
model_py = tf.keras.models.load_model(f"{path}/mlp_model.h5")

## Float TFLite model

In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_py)
tflite_model = converter.convert()

with open(path_to_save + "/model_float32.tflite", "wb") as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpkzgc3lu9/assets


INFO:tensorflow:Assets written to: /tmp/tmpkzgc3lu9/assets
2024-10-28 15:27:06.130078: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-10-28 15:27:06.130107: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-10-28 15:27:06.130239: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpkzgc3lu9
2024-10-28 15:27:06.130929: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2024-10-28 15:27:06.130950: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpkzgc3lu9
2024-10-28 15:27:06.134090: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2024-10-28 15:27:06.156214: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpkzgc3lu9
2024-10-28 15:27:06.166039: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

## Quantização de Faixa Dinâmica 

A quantização de faixa dinâmica é a mais simples e rápida, quantizando somente os pesos do modelo de ponto flutuante para inteiros de 8 bits.

In [22]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_py)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

with open(path_to_save + '/model_dynamic_range_quantized.tflite', 'wb') as f:
    f.write(tflite_quant_model)

INFO:tensorflow:Assets written to: /tmp/tmpnjefkhnq/assets


INFO:tensorflow:Assets written to: /tmp/tmpnjefkhnq/assets
2024-10-28 15:27:06.967565: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-10-28 15:27:06.967610: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-10-28 15:27:06.967806: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpnjefkhnq
2024-10-28 15:27:06.969315: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2024-10-28 15:27:06.969351: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpnjefkhnq
2024-10-28 15:27:06.973377: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2024-10-28 15:27:07.004984: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpnjefkhnq
2024-10-28 15:27:07.017524: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

## Quantização Float16

Esta configuração converte todos os pesos e ativações para float16, que é útil para dispositivos com suporte nativo a esse formato.

In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_py)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_model_float16 = converter.convert()

with open(path_to_save + '/model_float16_quantized.tflite', 'wb') as f:
    f.write(tflite_model_float16)

INFO:tensorflow:Assets written to: /tmp/tmppldcvnwj/assets


INFO:tensorflow:Assets written to: /tmp/tmppldcvnwj/assets
2024-10-28 15:27:07.613669: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-10-28 15:27:07.613717: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-10-28 15:27:07.613910: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmppldcvnwj
2024-10-28 15:27:07.615029: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2024-10-28 15:27:07.615072: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmppldcvnwj
2024-10-28 15:27:07.618546: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2024-10-28 15:27:07.658474: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmppldcvnwj
2024-10-28 15:27:07.669252: I tensorflow/cc/saved_model/loader.cc:301] SavedModel

## Quantização Inteira Completa

Esta abordagem quantiza todos os pesos e ativações para inteiros de 8 bits e requer um conjunto de dados representativo para calibrar a quantização.

In [24]:
path = '/home/murilo/workspace/tcc/tcc/python_model/database/x_train.csv'
x_train = pd.read_csv(path)

def representative_dataset_generator():
    for value in x_train.values:
        yield [np.array(value, dtype=np.float32, ndmin=2)]

In [25]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_py)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset_generator
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_model = converter.convert()

with open(path_to_save + '/model_full_integer_quantized.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpxdhz1bqq/assets


INFO:tensorflow:Assets written to: /tmp/tmpxdhz1bqq/assets
/home/murilo/workspace/tcc/tcc/python_model/.env/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2024-10-28 15:27:08.301090: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-10-28 15:27:08.301135: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-10-28 15:27:08.301325: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpxdhz1bqq
2024-10-28 15:27:08.302625: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2024-10-28 15:27:08.302660: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpxdhz1bqq
2024-10-28 15:27:08.305234: I tensor

## Quantização Somente Inteiro (Weight-only Quantization)

Quantiza somente os pesos para inteiros de 8 bits.

In [26]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_py)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_model_weights_only = converter.convert()

with open(path_to_save + '/model_weights_only_quantized.tflite', 'wb') as f:
    f.write(tflite_model_weights_only)

INFO:tensorflow:Assets written to: /tmp/tmpd9poejfy/assets


INFO:tensorflow:Assets written to: /tmp/tmpd9poejfy/assets
2024-10-28 15:27:08.970382: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2024-10-28 15:27:08.970415: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2024-10-28 15:27:08.970543: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpd9poejfy
2024-10-28 15:27:08.971151: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2024-10-28 15:27:08.971176: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: /tmp/tmpd9poejfy
2024-10-28 15:27:08.973387: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2024-10-28 15:27:09.002518: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpd9poejfy
2024-10-28 15:27:09.019618: I tensorflow/cc/saved_model/loader.cc:301] SavedModel